In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install cython
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install opencv-python-headless


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Now set your DATA_ROOT to wherever you put your folders:
DATA_ROOT = ""


In [ ]:
import os
from PIL import Image
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.structures import BoxMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset


In [ ]:
TRAIN_IMG_DIRS = [
    os.path.join(DATA_ROOT, ""), #datapath to training images of fog
    os.path.join(DATA_ROOT, ""), #datapath to training images of snow
]
TRAIN_LABEL_DIR = os.path.join(DATA_ROOT, "") #datapath to training labels

VAL_IMG_DIRS = [
    os.path.join(DATA_ROOT, ""), #datapath to validation images of fog
    os.path.join(DATA_ROOT, ""), #datapath to validation images of snow
]
VAL_LABEL_DIR = os.path.join(DATA_ROOT, "") #datapath to validation labels

TEST_IMG_DIR   = os.path.join(DATA_ROOT, "") #datapath to testing images
TEST_LABEL_DIR = os.path.join(DATA_ROOT, "") #datapath to test labels

CLASS_NAMES = [
    'ambulance','army vehicle','auto rickshaw','bicycle','bus',
    'car','garbagevan','human hauler','minibus','minivan',
    'motorbike','pickup','policecar','rickshaw','scooter',
    'suv','taxi','three wheelers -CNG-','truck','van','wheelbarrow'
]

In [ ]:

def get_yolo_dicts(img_dirs, label_dir):
    dicts = []
    for d in img_dirs:
        for fn in os.listdir(d):
            if not fn.lower().endswith((".jpg","png")): continue
            path = os.path.join(d, fn)
            w, h = Image.open(path).size
            rec = {
                "file_name": path,
                "image_id": f"{os.path.basename(d)}_{os.path.splitext(fn)[0]}",
                "height": h, "width": w
            }
            annos = []
            txt = os.path.join(label_dir, os.path.splitext(fn)[0] + ".txt")
            if os.path.exists(txt):
                for line in open(txt):
                    cid, cx, cy, rw, rh = map(float, line.split())
                    x0 = (cx - rw/2) * w
                    y0 = (cy - rh/2) * h
                    annos.append({
                        "bbox": [x0, y0, rw*w, rh*h],
                        "bbox_mode": BoxMode.XYWH_ABS,
                        "category_id": int(cid),
                    })
            rec["annotations"] = annos
            dicts.append(rec)
    return dicts

# Precompute & cache so DefaultTrainer starts instantly
train_dicts = get_yolo_dicts(TRAIN_IMG_DIRS, TRAIN_LABEL_DIR)
val_dicts   = get_yolo_dicts(VAL_IMG_DIRS,   VAL_LABEL_DIR)
test_dicts  = get_yolo_dicts([TEST_IMG_DIR], TEST_LABEL_DIR)

# Register splits
DatasetCatalog.register("street_train", lambda: train_dicts)
MetadataCatalog.get("street_train").set(thing_classes=CLASS_NAMES)
DatasetCatalog.register("street_val",   lambda: val_dicts)
MetadataCatalog.get("street_val").set(thing_classes=CLASS_NAMES)
DatasetCatalog.register("street_test",  lambda: test_dicts)
MetadataCatalog.get("street_test").set(thing_classes=CLASS_NAMES)


In [ ]:
# **Cell 4**: Configure & train on GPU (T4)
cfg = get_cfg()
cfg.merge_from_file(
    model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")
)
# Use COCO pre-trained weights
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(
    "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"
)

# Splits
cfg.DATASETS.TRAIN = ("street_train",)
cfg.DATASETS.TEST  = ()            # disable auto-validation during training

# Speed tweaks (moderate resolution)
cfg.INPUT.MIN_SIZE_TRAIN = (600,)  # default 800
cfg.INPUT.MAX_SIZE_TRAIN = 1000    # default 1333
cfg.INPUT.MIN_SIZE_TEST  = 600
cfg.INPUT.MAX_SIZE_TEST  = 1000

# DataLoader & Solver
cfg.DATALOADER.NUM_WORKERS              = 4
cfg.SOLVER.IMS_PER_BATCH                = 4
cfg.SOLVER.BASE_LR                      = 1e-4
cfg.SOLVER.MAX_ITER                     = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES         = len(CLASS_NAMES)

# Output dir
cfg.OUTPUT_DIR = "./output_fast_gpu"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Train!
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
# **Cell 5**: Evaluate on validation set with COCO metrics
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST    = ("street_test",)  # now enable val split

evaluator = COCOEvaluator("street_test", cfg, False, output_dir="./output_test")
val_loader = build_detection_test_loader(cfg, "street_test")

metrics = inference_on_dataset(trainer.model, val_loader, evaluator)
print("=== COCO metrics on street_test ===")
print(metrics)


In [ ]:
# === Make sure predictor is defined ===
import os
from detectron2.engine import DefaultPredictor

# Point to your final model checkpoint
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
# Only keep detections with score ≥ 0.5
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)


In [ ]:
# === Cell 6: Confusion Matrix + Example Prediction (with empty‐pred guard) ===

import os
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import cv2

from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# 0) (Re)define predictor if needed
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
predictor = DefaultPredictor(cfg)

# 1) Build y_true / y_pred by matching each GT box to the best pred box (IoU ≥ 0.5)
y_true, y_pred = [], []

def xywh_to_xyxy(b):
    x, y, w, h = b
    return [x, y, x + w, y + h]

for rec in val_dicts:
    img = cv2.imread(rec["file_name"])
    outputs = predictor(img)["instances"].to("cpu")

    # get predictions (may be empty)
    if len(outputs.pred_boxes) == 0:
        continue  # no predictions → skip this image entirely

    pred_boxes   = outputs.pred_boxes.tensor.numpy()  # shape [N,4] in XYXY
    pred_classes = outputs.pred_classes.numpy()       # shape [N]

    for ann in rec["annotations"]:
        gt_box = np.array(xywh_to_xyxy(ann["bbox"]))

        # Compute IoU of this GT with every pred
        ious = []
        for pb in pred_boxes:
            xi1 = max(gt_box[0], pb[0]); yi1 = max(gt_box[1], pb[1])
            xi2 = min(gt_box[2], pb[2]); yi2 = min(gt_box[3], pb[3])
            inter = max(0, xi2 - xi1) * max(0, yi2 - yi1)
            area_gt = (gt_box[2] - gt_box[0]) * (gt_box[3] - gt_box[1])
            area_pb = (pb[2] - pb[0]) * (pb[3] - pb[1])
            union = area_gt + area_pb - inter
            ious.append(inter / union if union > 0 else 0)

        ious = np.array(ious)
        best_i = ious.argmax()
        if ious[best_i] >= 0.5:
            y_true.append(ann["category_id"])
            y_pred.append(int(pred_classes[best_i]))
        # else: we skip this GT (treat as unmatched)

# 2) Compute & plot confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(CLASS_NAMES))))

plt.figure(figsize=(8,8))
plt.imshow(cm, aspect='auto')
plt.colorbar()
plt.xticks(range(len(CLASS_NAMES)), CLASS_NAMES, rotation=90)
plt.yticks(range(len(CLASS_NAMES)), CLASS_NAMES)
plt.title("Confusion Matrix (IoU ≥ 0.5 matches)")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.tight_layout()
plt.show()

# 3) Show one random prediction example
rec = random.choice(val_dicts)
img = cv2.imread(rec["file_name"])
outputs = predictor(img)["instances"].to("cpu")

v = Visualizer(
    img[:, :, ::-1],
    metadata=MetadataCatalog.get("street_val"),
    scale=1.0
)
out = v.draw_instance_predictions(outputs)

plt.figure(figsize=(8,6))
plt.imshow(out.get_image()[:, :, ::-1])
plt.axis("off")
plt.title(f"Example: {os.path.basename(rec['file_name'])}")
plt.show()


In [ ]:
# 2) Compute & plot confusion matrix with white→blue colormap
cm = confusion_matrix(y_true, y_pred, labels=list(range(len(CLASS_NAMES))))

plt.figure(figsize=(8,8))
plt.imshow(cm, aspect='auto', cmap='Blues')      # add cmap='Blues'
plt.colorbar()
plt.xticks(range(len(CLASS_NAMES)), CLASS_NAMES, rotation=90)
plt.yticks(range(len(CLASS_NAMES)), CLASS_NAMES)
plt.title("Confusion Matrix (IoU ≥ 0.5 matches)")
plt.xlabel("Predicted Class")
plt.ylabel("True Class")
plt.tight_layout()
plt.show()
